- Recombine the final sequece and croos over between the weak links -> needed a custum function, maybe full deterministic
- Stats and dataframe for the SNP and other variants
- Implement gaps NNNNN

In [1]:
from Bio import pairwise2
from Bio import SeqIO
import yaml
from math import modf
from Bio.Seq import Seq
from inspyred import swarm
from inspyred import benchmarks
from inspyred import ec
from inspyred.ec import selectors
from collections import deque
from itertools import combinations
import random
import pandas as pd
import seaborn as sns
from multiprocessing import Pool
from tqdm import tqdm
import tensorflow as tf
import collections
from itertools import combinations
from numba import njit
import random
import numpy as np
from multiprocessing import Pool
from joblib import Parallel, delayed
import seaborn as sns
import torch as tr
import time


In [22]:
# Implementation using tensors

read_1 = "ATTTAggggTA"
read_1 = read_1.upper()

read_2 = "ATTCGGATCGA"
read_2 = read_2.upper()

read_tf_2 = tf.constant([ord(c) for c in read_2])
read_tf_1 = tf.constant([ord(c) for c in read_1])
read_tf_1.get_shape()[0]
# read_tf = tf.strings.unicode_encode(read_tf, "UTF-8")

for i in range(read_tf_1.get_shape()[0]):
    print(i)

TypeError: 'float' object cannot be interpreted as an integer

### Here trying parallelization with Numba e prange

In [11]:
import collections
from numba import jit, prange, vectorize
from joblib import Parallel, delayed
import random
import numpy as np
from tqdm import tqdm
from Bio import SeqIO
import seaborn as sns
import torch as tr
import time

seq = ""
for seq_record in SeqIO.parse("C:\\Users\\filoa\\Desktop\\Programming_trials\\Assembler\\Data\\GCA_014117465.1_ASM1411746v1_genomic.fna", format = "fasta"):
    seq += str(seq_record.seq.upper())

def comstum_reads(seq: str, length_reads = 10, coverage = 5, verbose = False) -> list:
    
    """The function split the sequence in input in reads.
    The splitting is done using random numbers, the amount of reds is given by: (len(seq)/length_read)*coverage.
    """

    number_of_reads = int(len(seq)/length_reads) * coverage
    starting_pos = random.sample(range(0, len(seq)-length_reads+1), number_of_reads)
    reads = []

    for num in starting_pos:
        reads.append(seq[num:num+length_reads])

    if verbose == True:
        # This part has the only aim to show some stats on the reads
        y = [0 for i in range(0,len(seq)+1)]
        for i in starting_pos:
            for j in range(i, i+length_reads+1):
                y[j] += 1 
        sns.set_theme(style="darkgrid")
        sns.lineplot(y)
        print(f"There are {y.count(0)} bases that have 0 coverage.")

    return reads

def tensor_score(ord_sequence:list, zeros:True)->int:
    raise NotImplemented

def tensor_align(sequence_one:str, sequence_two:str):
    raise NotImplemented

def uni_code(read:str)->np.ndarray:
    return np.array([ord(c) for c in read])

reads = comstum_reads(seq[:3000], length_reads = 150, coverage = 6)
reads = [uni_code(read) for read in reads]

### Vectorize e Numba

@vectorize()
def np_score(align_list: np.ndarray, zeros = True)->float:
    """This function is a replacement for the np function np.count_nonzero(), since inside the np_eval_function was needed to count the number of zeros (=matches).
    However this function raise an error when run with the numba decorator.
    """
    length = len(align_list)
    cnt = 0

    for i in align_list:
        if i == 0:
            cnt += 1

    if zeros:
        return float(cnt)
    else:
        return float(length-cnt)

@jit(parallel=True)
def np_align_func(seq_one:np.ndarray, seq_two:np.ndarray, match:int = 3, mismatch:int = -2) -> tuple:
    """This function is a replacement for the align function pirwise2.align.localms of the Bio library. This substitution has the aim of tackling the computational time of the
    eval_alignment function. In order to decrease the time, there was the need to create a compilable function with numba, which was also capable of being parallelised.
    As you can clearly see the function takes in input only the match and mismatch, because in this usage the gap introduction is useless.

    seq_one, seq_two = input sequences already trasformed in byte
    match, mismatch = integer value for the alignment

    Note: the mismatch should be negative
    Ex output: (12.0, 34, True)
    """

    # Initialization of outputs, the output is a tuple that contains: score of the alignment, a number indicating how the two reads align
    # and if the two sequences have been inverted in order during the process
    score = float(0)
    diff = 0
    switch = False

    # Since knowing which one is the longest is needed 
    if seq_one.shape[0] >= seq_two.shape[0]:
        max_lenght_seq = seq_one.shape[0]
        min_length_seq = seq_two.shape[0]

    else:
        switch = True
        max_lenght_seq = seq_two.shape[0]
        min_length_seq = seq_one.shape[0]
        seq_one, seq_two = seq_two, seq_one
    
    # Number of iterations
    num_iteration_int = (max_lenght_seq + min_length_seq - 1) // 2
    num_iteration = (max_lenght_seq + min_length_seq - 1) / 2
    alone = False

    if num_iteration > num_iteration_int:
        alone = True

    for i in prange(num_iteration_int):
        if i < min_length_seq:

            align_forw = seq_one[:(i+1)] - seq_two[-(i+1):]
            align_back = seq_two[:(i+1)] - seq_one[-(i+1):]

            cnt = 0
            for j in align_forw, align_back:
                part_score = np_score(j)*match + np_score(j, zeros=False)*mismatch

                if part_score >= score:
                    score = part_score
                    if cnt > 0:
                        diff = max_lenght_seq -i -1
                    else:
                        diff = -(min_length_seq -i -1)
                cnt += 1
        
        if i >= min_length_seq:
            align_forw = seq_one[i-min_length_seq+1:(i+1)] - seq_two[-(i+1):]
            align_back = seq_one[-(i+1):-(i-min_length_seq+1)] - seq_two[:(i+1)]

            cnt = 0
            for j in align_forw, align_back:
                part_score = np_score(j)*match + np_score(j, zeros=False)*mismatch

                
                if part_score >= score:
                    score = part_score
                    if cnt > 0:
                        diff = max_lenght_seq -i -1
                    else:
                        diff = -(min_length_seq -i -1)
                cnt += 1 

        if i == (num_iteration_int - 1) and alone:
            i += 1

            align_forw = seq_one[i-min_length_seq+1:(i+1)] - seq_two[-(i+1):]
            part_score = np_score(j)*match + np_score(j, zeros=False)*mismatch

            if part_score >= score:
                score = part_score
                diff = max_lenght_seq -i -1


    return (score, diff, switch)


def eval_allign_np(reads:list, par:list = [3, -2]) -> np.ndarray:
    """Funtion that evaulate the alignment

    reads: list of DNA sequences, each of the read is a list of integers that resemble the real sequence

    par: list of parameters to performe the alignment
    es (the examples represent the defoult parameters):
    match_score = 3,
    mismatch_penalty = -2,

    output:
    Matrix with the weigts (distances) between the reads (nodes)
    In this matrix there are both the scores of the alignment, recognizable for the tipical integer score (even if is a float point) and
    a flaot number (like 0.23) which is needed after to recompose the sequence; it indicates the overlapping bases.
    Ex:
        allignment score -> 2.0, 13.0, ...
        overlapping number -> 0.241, 0.61, 0.561, ...
            To avoid problem later with 0 a 1 digit is added for then remove it. So 0.30 become 0.301 but the corret indices are 12 and 30.

        These two numbers are link by the position in the matrix which are the trasposition
        Score 14.0 in position (1,5) --> 0.34 in position (5,1). Only the score position is referred
        to the direction of the edge.
        1 ---> 5 with allignment score 14 and read_1 is overlapped with read_5 in positions 34 (both included)

    Example of a matrix with three reads:

        | 1    | 2    | 3    
     1  | 0    |3.0   | 0.231 
     2  | 0.601|  0   | 23.0
     3  | 18.0 | 0.701|  0
    """
    length = len(reads)
    # initialization of the matrices
    weigth_matrix = np.zeros((length, length))

    # The score of the allingment of read[1] to read[2] is the same of the opposite (read[2] to read[1])
    # So when the function found the diretionality of the allignment put the score in rigth spot and a 0 in the wrong one.
    visited = collections.deque([j for j in range(length)])
    # comb = combinations(range(len(reads)),2)
    # for i,j in comb:

    for i in tqdm(range(length)):

        for j in visited:

            if i == j:
                # the diagonal of the matrix has 0 score because we are allinging the same sequence
                continue
            else:
                # pairwise must return a positive score, if there is no one it return None
                alignment = np_align_func(reads[i], reads[j], match = par[0], mismatch = par[1])

                if alignment[2]:
                    if alignment[1] > 0:
                        weigth_matrix[j, i] = alignment[0]
                        weigth_matrix[i, j] = float(f"{0}.{abs(alignment[1])}1")
                    
                    else:
                        weigth_matrix[i, j] = alignment[0]
                        weigth_matrix[j, i] = float(f"{0}.{abs(alignment[1])}1")

                else:
                    if alignment[1] > 0:
                        weigth_matrix[i, j] = alignment[0]
                        weigth_matrix[j, i] = float(f"{0}.{abs(alignment[1])}1")
                    
                    else:
                        weigth_matrix[j, i] = alignment[0]
                        weigth_matrix[i, j] = float(f"{0}.{abs(alignment[1])}1")

                    
        visited.popleft()
    return weigth_matrix

# eval_allign_np(reads = reads)

### Parallel and delayed

@jit(nopython = True)
def np_score(align_list: np.ndarray, zeros = True)-> int:
    """This function is a replacement for the np function np.count_nonzero(), since inside the np_eval_function was needed to count the number of zeros (=matches).
    However this function raise an error when run with the numba decorator.
    """
    length = len(align_list)
    cnt = 0

    for i in align_list:
        if i == 0:
            cnt += 1

    if zeros:
        return cnt
    else:
        return length-cnt


@jit(nopython=True)
def np_align_func(seq_one:np.ndarray, seq_two:np.ndarray, match:int = 3, mismatch:int = -2) -> tuple:
    """This function is a replacement for the align function pirwise2.align.localms of the Bio library. This substitution has the aim of tackling the computational time of the
    eval_alignment function. In order to decrease the time, there was the need to create a compilable function with numba, which was also capable of being parallelised.
    As you can clearly see the function takes in input only the match and mismatch, because in this usage the gap introduction is useless (for the moment).
    This function return only the BEST alignment.

    seq_one, seq_two = input sequences already trasformed in integers by ord function
    match, mismatch = integer value for the alignment

    Note: the mismatch should be negative
    Output: A tuple with the alignment score, a number that resamble the shift of the alignemnt, and a boolean which indicates if the order
        of the input has been inverted or not. This last element is essential to retrive the order, so which of the two will be place before the other one.
    Ex output: (12.0, 34, True)
    """

    # Initialization of outputs, the output is a tuple that contains: score of the alignment, a number indicating how the two reads align
    # and if the two sequences have been inverted in order during the process
    score = 0
    diff = 0
    switch = False

    # Since knowing which one is the longest is needed for the epoch
    if seq_one.shape[0] >= seq_two.shape[0]:
        max_lenght_seq = seq_one.shape[0]
        min_length_seq = seq_two.shape[0]

    else:
        switch = True
        max_lenght_seq = seq_two.shape[0]
        min_length_seq = seq_one.shape[0]
        seq_one, seq_two = seq_two, seq_one
    
    # Number of iterations (N + n -1)/2 because each iteration is producing two alignment one confronting the sequenvces from the forward
    # and from the backward
    num_iteration_int = (max_lenght_seq + min_length_seq - 1) // 2
    num_iteration = (max_lenght_seq + min_length_seq - 1) / 2
    alone = False # There could be needed an extra iteration if this (N + n -1)/2 is odd 

    if num_iteration > num_iteration_int:
        alone = True

    for i in range(num_iteration_int):
        if i < min_length_seq:

            # Back/Forward alignments, only overlapping bases are being used
            align_forw = seq_one[:(i+1)] - seq_two[-(i+1):]
            align_back = seq_two[:(i+1)] - seq_one[-(i+1):]

            cnt = 0
            for j in align_forw, align_back:
                part_score = np_score(j)*match + np_score(j, zeros=False)*mismatch

                if part_score >= score:
                    score = part_score
                    if cnt > 0:
                        # If the diff value is positive the first sequence is upstream
                        # The index diff is included
                        diff = max_lenght_seq -i -1
                    else:
                        # If the diff value is negative the second sequence if the one upstream
                        # The index diff is included
                        diff = -(min_length_seq -i -1)
                cnt += 1
        
        if i >= min_length_seq:
            align_forw = seq_one[i-min_length_seq+1:(i+1)] - seq_two[-(i+1):]
            align_back = seq_one[-(i+1):-(i-min_length_seq+1)] - seq_two[:(i+1)]

            cnt = 0
            for j in align_forw, align_back:
                part_score = np_score(j)*match + np_score(j, zeros=False)*mismatch

                
                if part_score >= score:
                    score = part_score
                    if cnt > 0:
                        diff = max_lenght_seq -i -1
                    else:
                        diff = -(min_length_seq -i -1)
                cnt += 1 

        if i == (num_iteration_int - 1) and alone:
            i += 1

            align_forw = seq_one[i-min_length_seq+1:(i+1)] - seq_two[-(i+1):]
            part_score = np_score(j)*match + np_score(j, zeros=False)*mismatch

            if part_score >= score:
                score = part_score
                diff = max_lenght_seq -i -1


    return (score, diff, switch)


def eval_allign_np(reads:list, par:list = [3, -2]) -> np.ndarray:
    """Funtion that evaulate the alignment

    reads: list of DNA sequences, each of the read is a list of integers that resemble the real sequence

    par: list of parameters to performe the alignment
    es (the examples represent the defoult parameters):
    match_score = 3,
    mismatch_penalty = -2,

    output:
    Matrix with the weigts (distances) between the reads (nodes)
    In this matrix there are both the scores of the alignment, recognizable for the tipical integer score (even if is a float point) and
    a flaot number (like 0.23) which is needed after to recompose the sequence; it indicates the overlapping bases.
    Ex:
        allignment score -> 2.0, 13.0, ...
        overlapping number -> 24.1, 6.1, 56.1, ...
            To avoid problem later with 0 a 1 digit is added for then remove it. So 0.30 become 0.301 but the corret indices are 12 and 30.

        These two numbers are link by the position in the matrix which are the trasposition
        Score 14.0 in position (1,5) --> 34.1 in position (5,1). Only the score position is referred
        to the direction of the edge.
        1 ---> 5 with allignment score 14 and read_1 is overlapped with read_5 in positions 34 (both included)

    Example of a matrix with three reads:

        | 1    | 2    | 3    
     1  | 0    |3.0   | 23.1 
     2  | 60.1 |  0   | 23.0
     3  | 18.0 | 70.1 |  0
    """
    length = len(reads)
    # initialization of the matrices
    weigth_matrix = np.zeros((length, length))

    # The score of the allingment of read[1] to read[2] is the same of the opposite (read[2] to read[1])
    # So when the function found the diretionality of the allignment put the score in rigth spot and a 0 in the wrong one.
    visited = collections.deque([j for j in range(length)])
    # comb = combinations(range(len(reads)),2)
    # for i,j in comb:

    for i in tqdm(range(length)):

        for j in visited:

            if i == j:
                # the diagonal of the matrix has 0 score because we are allinging the same sequence
                continue
            else:
                # pairwise must return a positive score, if there is no one it return None
                alignment = np_align_func(reads[i], reads[j], match = par[0], mismatch = par[1])

                if alignment[0] > 0:

                    if alignment[2]:
                        # Swithch happend so reads[j] is longer then reads[i]
                        if alignment[1] > 0:
                            # cond = first sequence is upstream
                            weigth_matrix[j, i] = alignment[0]
                            weigth_matrix[i, j] = float(f"{abs(alignment[1])}.1")
                        
                        else:
                            # cond = first sequence is downstream
                            weigth_matrix[i, j] = alignment[0]
                            weigth_matrix[j, i] = float(f"{abs(alignment[1])}.1")

                    else:
                        if alignment[1] > 0:
                            # cond = first sequence is upstream
                            weigth_matrix[i, j] = alignment[0]
                            weigth_matrix[j, i] = float(f"{abs(alignment[1])}.1")
                        
                        else:
                            # cond = first sequence is downstream
                            weigth_matrix[j, i] = alignment[0]
                            weigth_matrix[i, j] = float(f"{abs(alignment[1])}.1")

                else:
                    continue

                    
        visited.popleft()
    return weigth_matrix


##########
comb = combinations(range(len(reads)),2)

real_comb = [(reads[i[0]], reads[i[1]]) for i in comb]

results = Parallel(n_jobs=6)(delayed(np_align_func_par)(i) for i in real_comb)


UnsupportedRewriteError: [1mFailed in nopython mode pipeline (step: convert to parfors)
[1mOverwrite of parallel loop index
[1m
File "..\..\..\..\AppData\Local\Temp\ipykernel_19680\485280851.py", line 331:[0m
[1m<source missing, REPL/exec in use?>[0m
[0m[0m

### Changing the final reconstruction function
- return matrix
- paralleize the coding/decoding in number
- make a separe function for the consensus
- probably is better to make lower the memory usage, since the consensus matrix will be in the same order of the length

In [ ]:
def final_consensus(path:list, reads:list, positions:list, length:int, max_coverage: int = 16, verbose:bool = False) ->str:
    """This function create a matrix and write down the numbers resembling the path found by the ants algorithm
    """
    #Diff is included

    cons_matrix = np.zeros((max_coverage, length))
    leng = len(cons_matrix[0])
    cum_dif = 0
    adding = np.zeros((max_coverage, int(length/100)))

    for i,j in path:
        # Here i,j represent the edge of the graph, to retrive not the score but the alignment
        # the function needs the opposite position where there are these informations matrix[j][i]

        num = str(positions[j][i]).split(".")
        dif = int(num[0])

        if cons_matrix[0,0] == 0:
            # This first part is for starting the writing of the matrix
            
            for pos in range(0, len(reads[i]) + 1):
                if cons_matrix[0, pos] != 0:
                    cons_matrix = np.append(cons_matrix, adding, 1)
                cons_matrix[0, pos] = reads[i][pos]
            cum_dif += dif
            temp = 0
            for p in range(cum_dif + 1, cum_dif + len(reads[j]) + 1): # added +1 in last mod
                if cons_matrix[1,pos] != 0:
                    cons_matrix = np.append(cons_matrix, adding, 1)
                cons_matrix[1, p] = reads[j][temp]
                temp += 1

        else:
            # There is a check if the initialized matrix is big enough to contain all tha bases, columns wise
            if cons_matrix.shape[1] < cum_dif + len(reads[j])*2:
                cons_matrix = np.append(cons_matrix, adding, 1)
            else:
                cum_dif += dif
                temp = 0
                for pos in range(cum_dif + 1, cum_dif + len(reads[j]) + 1): # added +1 in last mod
                    row = 0
                    while cons_matrix[row, pos] >= 1:
                        row += 1
                    # There is a check if the initialized matrix is big enough to contain all tha bases, row wise
                    if row == cons_matrix.shape[0]:
                        cons_matrix = np.append(cons_matrix, np.zeros((2, cons_matrix.shape[1])) ,0)
                    cons_matrix[row, pos] = reads[j][temp]
                    temp +=1

    return cons_matrix


def re_build(cons_matrix:list):
    
    dictionary = "ATCG"
    cons_seq = ""
    for i in range(0, len(cons_matrix)):
        base = [x for x in cons_matrix[:,i] if x > 0]
        if base == []:
            return cons_seq
        ind = []
        for num in [ord(c) for c in dictionary]:
            ind.append(base.count(num))
        more_frequent = ind.index(max(ind))
        # TODO stats
        cons_seq += dictionary[more_frequent]

    return cons_seq


def join_consensus_sequence(consensus_matrix:np.ndarray, cpus:int=1)-> str:
    "This functio is just to implement the use of multiples core for recostructing the final sequence."

    step = len(consensus_matrix)/cpus
    cnt = 0
    partials = []
    for i in range(step, len(consensus_matrix) + step, step):
        partials.append((cnt,i))
        cnt += i
    sub_parts = [consensus_matrix[:,i:j] for i,j in partials]
    
    results = Parallel(n_jobs=cpus)(delayed(re_build)(i) for i in sub_parts)

    return "".join(results)


### Parallelization of the string of bases

In [6]:
from joblib import Parallel, delayed
import numpy as np
from numba import jit

def uni_code(read:str)->np.ndarray:
    return np.array([ord(c) for c in read])

def de_code(read:np.ndarray)->str:
    return "".join([chr(c) for c in read])

reads = ["ATTA", "ATTAA", "ATTCGAT", "GGAGAGTCGGA"]

print(de_code(uni_code(reads[0])))

results = Parallel(n_jobs=5)(delayed(uni_code)(i)for i in reads)
print(results)

re_results = Parallel(n_jobs=5)(delayed(de_code)(j)for j in results)
print(re_results)

ATTA
[array([65, 84, 84, 65]), array([65, 84, 84, 65, 65]), array([65, 84, 84, 67, 71, 65, 84]), array([71, 71, 65, 71, 65, 71, 84, 67, 71, 71, 65])]
['ATTA', 'ATTAA', 'ATTCGAT', 'GGAGAGTCGGA']


### Assembly class problem

In [33]:
class Assembly_problem():
    """Defines the de novo genome assembly problem.
    
    This class based on the Traveling Salesman problem defines the problem
    of assembling a new genome for which no reference is available (de novo assembly):
    given a set of genomic reads and their pairwise overlap score, find the
    path generating the longest consensus sequence. This problem assumes that 
    the ``weights`` parameter is an *n*-by-*n* matrix of pairwise 
    overlap among *n* reads. This problem is treated as a 
    maximization problem, socfitness values are determined to be the 
    proportional to the sum of the overlaps between each couple of reads
    (the weight of the edge) and the length of the final assembled sequence.
    
    Public Attributes:c
    
    - *weights* -- the two-dimensional list of pairwise overlap 
    - *components* -- the set of ``TrailComponent`` objects constructed
      from the ``weights`` attribute, where the element is the ((source,
      destination), weight)
    - *bias* -- the bias in selecting the component of maximum desirability
      when constructing a candidate solution for ant colony optimization 
      (default 0.5)
    """
    
    def __init__(self, reads, approximate_length):
        self.weights = eval_allign(reads)
        self.reads = reads
        self.components = [swarm.TrailComponent((i, j), value=(self.weights[i][j])) for i, j in itertools.permutations(range(len(self.weights)), 2) if modf(self.weights[i,j])[0] == 0]
        self.bias = 0.5
        self.bounder = ec.DiscreteBounder([i for i in range(len(self.weights))])
        self.best_path = None
        self.maximize = True
        self.length = approximate_length

    def generator(self, random, args):
            """Return a candidate solution for an evolutionary computation."""
            locations = [i for i in range(len(self.weights))]
            random.shuffle(locations)
            return locations
    
    def constructor(self, random, args):
        """Return a candidate solution for an ant colony optimization."""
        self._use_ants = True
        candidate = []
        feasible_components = [1]   #Fake initialization to allow while loop to start
        
        # We need to visit all the nodes that CAN be visited, the graph is directed and not complete, meaning we can have no more nodes to visit without visiting all the
        # nodes in the graph, thus, our termination condition is not visitin all the nodes but not having anymore feasible components
        while len(feasible_components) > 0:
            # At the start of the visit, all the components are feasible
            if len(candidate) == 0:
                feasible_components = self.components
            elif len(candidate) == len(self.weights) - 1: # All the nodes have been visited
                return candidate
            else:
                # Update feasible components and set of already visited nodes considering the node visited in the last iteration
                last = candidate[-1]
                already_visited = [c.element[0] for c in candidate]
                already_visited.extend([c.element[1] for c in candidate])
                already_visited = set(already_visited)
                feasible_components = [c for c in self.components if c.element[0] == last.element[1] and c.element[1] not in already_visited]
            if len(feasible_components) == 0:
                return candidate
            # Choose a feasible component
            if random.random() <= self.bias:
                next_component = max(feasible_components)
            else:
                next_component = selectors.fitness_proportionate_selection(random, feasible_components, {'num_selected': 1})[0]
            candidate.append(next_component)
        return candidate
    
    def cross_over(path:list, matrix:list)->list:
        """This function recombine the solution. Takes the path and the score associated to each edge
        iterate over the path and switch two edge.
        """
        imaginary_string = range(len(path))

        min_1 = path.index(min([c.value for c in path]))
        min_2 = path.index(min([c.value for c in path if (c.element[0] == min_1[0]) and (c.element[1] == min_1[1])]))
        return None
    
    def evaluator(self, candidates, args):
        """Return the fitness values for the given candidates."""
        # TODO use normal distribution
        fitness = []
        for candidate in candidates:
            total = 0
            current_path = []
            for c in candidate:
                total += self.weights[c.element[0]][c.element[1]]
                current_path.append(c)
            # last = (candidate[-1].element[1], candidate[0].element[0])
            # current_path=[(i.element[0],i.element[1]) for i in candidate] # maybe i is enough
            # total += self.weights[last[0]][last[1]]
            current_sequence = consensus_sequence(current_path, reads=self.reads, positions=self.weights, length=self.length)
            length_score = abs((self.length-current_sequence)/self.length)
            s = [5, 3, 1, 0.5, 0.2]
            perc=[0, 0.01, 0.05, 0.08, 0.1, 0.2]
            l_score = 0.1
            for i in range(len(perc)-1):
                if length_score >= perc[i] and length_score < perc[i+1]:
                    l_score = s[perc.index(perc[i])]

            if self.best_path == None:
                self.best_path = current_path
            
            score = total * l_score
            fitness.append(score)

        return fitness

### Align function

### Task Graph

In [19]:
# Simplification of the graph
from scipy.io import loadmat
from joblib import Parallel, delayed, cpu_count
from numba import jit, prange
import numpy as np


@jit(nopython = True)
def __prepare_simpl_intup__(matrix:np.ndarray)->list:
    """Is needed to set up the parallelization, divide the matrix in columns
    """
    # output = (array, len_array, column)

    len_arrray = matrix.shape[0]
    my_list = []

    for i in range(len(matrix)):
        my_list.append((list(matrix[:,i]), len_arrray, i))

    return my_list


def __matrix_selection__(input_tuple:tuple, cut_off = 0.2)->list:
    """Value the distibution of the columns, in this way select the ones
    above the third quantile
    """

    # Init
    array = input_tuple[0]
    array_len = input_tuple[1]
    column = input_tuple[2]

    # DO DO
    links = [x for x in array if (x > 0) and (str(x).split(".")[1] == "0")]
    chosen = sorted(links)[-(int(len(links)*cut_off)):]
    print(f"{links}\n{len(links)}\n{chosen}\n{len(chosen)}")
    dissmissable_links = []

    for i in links:
        if i not in chosen:
            dissmissable_links.append((array.index(i), column))

    return dissmissable_links


def __matrix_sempl__(matrix:np.ndarray, dissmissable_links:list) -> np.ndarray:
    """Changes the occurrencie valued as unprobable in zeros, eraising in this way the link
    """
    for epoch in dissmissable_links:
        for i,j in epoch:
            matrix[i,j] = 0. 
            matrix[j,i] = 0.

    return matrix


def graph_semplification(graph:np.ndarray, cores:int)->np.ndarray:
    
    list_of_tuple = __prepare_simpl_intup__(graph)

    indeces_to_cut = Parallel(n_jobs = cores)(delayed(__matrix_selection__)(i) for i in list_of_tuple)

    matrix = __matrix_sempl__(graph, dissmissable_links=indeces_to_cut)

    return matrix


matrix = loadmat("C:\\Users\\filoa\\Desktop\\Programming_trials\\Assembler\\Data\\graph_metadata.mat")["data"]

a = __prepare_simpl_intup__(matrix)

b = __matrix_selection__(a[0])

[15.0, 8.0, 7.0, 11.0, 10.0, 6.0, 15.0, 9.0, 23.0, 9.0, 6.0, 8.0, 3.0, 24.0, 8.0, 6.0, 21.0, 12.0, 9.0, 11.0, 11.0, 13.0, 10.0, 18.0, 15.0, 10.0, 15.0, 7.0, 11.0, 9.0, 4.0, 294.0, 12.0, 13.0, 7.0, 12.0, 12.0, 6.0, 11.0, 6.0, 12.0, 16.0, 13.0, 14.0, 10.0, 14.0, 9.0, 13.0, 11.0, 9.0, 17.0, 6.0, 17.0, 8.0, 9.0, 9.0, 14.0, 9.0, 3.0, 12.0, 10.0, 3.0, 7.0, 15.0, 9.0, 11.0, 8.0, 3.0, 114.0, 10.0, 11.0, 14.0, 4.0, 8.0, 9.0, 2.0, 4.0, 6.0, 13.0, 11.0, 6.0, 21.0, 7.0, 8.0, 7.0, 13.0, 6.0, 9.0, 17.0, 4.0, 13.0, 4.0, 13.0, 3.0, 19.0, 20.0, 5.0, 6.0, 16.0, 13.0, 3.0, 8.0, 4.0, 3.0, 11.0, 8.0, 11.0, 20.0, 5.0, 6.0, 4.0, 12.0, 5.0, 7.0, 9.0, 16.0, 13.0, 18.0, 3.0, 8.0, 6.0, 8.0, 14.0, 6.0, 9.0, 8.0, 6.0, 3.0, 14.0, 10.0, 15.0, 8.0, 4.0, 8.0, 12.0, 14.0, 25.0, 7.0, 7.0, 13.0, 8.0, 13.0, 9.0, 7.0, 339.0, 6.0, 10.0, 6.0, 11.0, 13.0, 6.0, 4.0, 8.0, 7.0, 16.0, 9.0, 12.0, 7.0, 11.0, 408.0, 15.0, 9.0, 10.0, 11.0, 7.0, 17.0, 4.0, 9.0, 6.0, 14.0, 6.0, 6.0, 4.0, 7.0, 3.0, 8.0, 10.0, 6.0, 3.0, 453.0, 10.0, 7.0,

In [26]:
import numpy as np
l = np.array([1,2,3,4,5,6])
list(l)


[1, 2, 3, 4, 5, 6]